In [3]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import subprocess
import urllib.request
# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"
a, b, c, d, e, f, g, h, i, j, k, l, m = sym.symbols('a b c d e f g h i j k l m')
n, o, p, q, r, s, t, u, v, w, x, y, z = sym.symbols('n o p q r s t u v w x y z')
symbol_list = (a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v,
w, x, y, z)
A, B, C, D, E, F, G, H, I, J, K, L, M = sym.symbols('A B C D E F G H I J K L M')
N, O, P, Q, R, S, T, U, V, W, X, Y, Z = sym.symbols('N O P Q R S T U V W X Y Z')
symbol_list = (A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, 
               V, W, X, Y, Z)
#%matplotlib notebook #incompatible with mpmath

In [7]:
https://www-nds.iaea.org/relnsd/vcharthtml/api_v0_notebook.html
def lc_read_csv(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

In [9]:
df = lc_read_csv(livechart + "fields=decay_rads&nuclides=241am&rad_types=g")
df

,energy,unc_en,intensity,unc_i,start_level_energy,end_level_energy,multipolarity,mixing_ratio,unc_mr,conversion_coeff,...,decay_%,unc_d,q,unc_q,d_z,d_n,d_symbol,ensdf_publication_cut-off,ensdf_authors,Extraction_date
0,13.8100,0.02,,,281.363,267.566,,,NaN,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
1,26.3446,0.0002,2.27,0.12212817037850031,59.54090,33.19632,E1,,NaN,8,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
2,27.0200,0.007,,,102.959,75.900,,,NaN,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
3,31.4000,,,,546.12,514.20,,,NaN,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
4,32.1830,,0.0174,0.00043620637317673384,,,,,NaN,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,97.0780,,0.001157241495610924,0.00003060653032015789,,,,,,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
175,101.0680,,0.001842154561621974,0.00004816628250112411,,,,,,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
176,114.1600,,0.0006717397419844511,0.000020932732130072342,,,,,,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
177,115.6080,,0.0009028182132271023,0.00002793238516187983,,,,,,,...,100,,5637.82,12,93,144,Np,15-Mar-2006,M. S. Basunia,2022-05-03
